In [ ]:
import cv2
import mediapipe as mp
import time
import tensorflow as tf
import numpy as np
import os
from collections import Counter
from tensorflow.keras.preprocessing.image import img_to_array
import math
from cvzone.ClassificationModule import Classifier

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
pTime = 0
imgSize = 48  
index = 0
displaySize = 200
frame_count = 0
current_prediction = None  
confidence_threshold = 0.9

In [ ]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=0.9,
    min_tracking_confidence=0.7
)
mpDraw = mp.solutions.drawing_utils
mpDrawingStyles = mp.solutions.drawing_styles

In [ ]:
def fps_calculation(img, pTime):
    cTime = time.time()
    fps = int(1 / (cTime - pTime))
    pTime = cTime
    cv2.putText(
        img, f"FPS:{fps}", (10, 70), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2
    )
    return pTime

In [ ]:
model = tf.keras.models.load_model("signlanguage1.h5")

In [ ]:
class_labels = ['AAROHAN', 'I', 'am', 'are', 'fine', 'hello', 'how', 'to', 'welcome', 'you']

In [ ]:
def show_pred(img, index, x_min, y_min, confidence=None):
    if 0 <= index < len(class_labels):
        confidence_text = f" ({confidence:.2f})" if confidence is not None else ""
        cv2.putText(
            img,
            f"Sign: {class_labels[index]}{confidence_text}",
            (x_min, y_min - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
        )

In [ ]:
def modify(imgWhite):
    if imgWhite is None or imgWhite.size == 0:
        return None
    
    imgGray = cv2.cvtColor(imgWhite, cv2.COLOR_BGR2GRAY)
    imgGray = cv2.resize(imgGray, (50, 50)) 
    imgGray = imgGray.astype("float32") / 255.0 
    imgGray = np.expand_dims(imgGray, axis=-1) 
    imgGray = np.expand_dims(imgGray, axis=0) 


    return imgGray

In [ ]:
def crop_hand_from_image(img, handLms):
    """Crop the hand region from the image using hand landmarks."""
    h, w, c = img.shape
    x_min, y_min = w, h
    x_max, y_max = 0, 0

    for lm in handLms.landmark:
        x, y = int(lm.x * w), int(lm.y * h)
        x_min = min(x, x_min)
        y_min = min(y, y_min)
        x_max = max(x, x_max)
        y_max = max(y, y_max)

    # Add padding
    padding = 40
    x_min = max(0, x_min - padding)
    y_min = max(0, y_min - padding)
    x_max = min(w, x_max + padding)
    y_max = min(h, y_max + padding)

    # Crop the image
    cropped_img = img[y_min:y_max, x_min:x_max]

    return cropped_img, (x_min, y_min, x_max, y_max)

In [ ]:
while True:
    success, img = cap.read()
    imgRgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRgb)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:

            # Get cropped hand
            cropped_img, bbox = crop_hand_from_image(img, handLms)

            if cropped_img.size == 0:
                print("Warning: Empty cropped image. Skipping frame.")
                continue

            # operations on cropped hand
            if cropped_img.size != 0:

                # Draw bounding box and landmarks
                x_min, y_min, x_max, y_max = bbox
                cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

                h, w = cropped_img.shape[:2]
                imgCropShape = cropped_img.shape
                imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

                aspectRatio = h / w
                if aspectRatio > 1:
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(cropped_img, (wCal, imgSize))
                    imgResizeShape = imgResize.shape
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhite[0 : imgResizeShape[0], wGap : wCal + wGap] = imgResize

                else:
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(cropped_img, (imgSize, hCal))
                    imgResizeShape = imgResize.shape
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhite[hGap : hCal + hGap, 0 : imgResizeShape[1]] = imgResize
                
                imgWhiteDisplay = imgWhite.copy()
                imgWhite = modify(imgWhite)
                if imgWhite is not None:
                    
                    prediction = model.predict(imgWhite) 
                    confidence = np.max(prediction)
                    index = np.argmax(prediction) 
                    print(f"Prediction: {prediction} | Index: {index} | Confidence: {confidence:.2f}")
                    if confidence >= confidence_threshold:
                      show_pred(img, index, bbox[0], bbox[1], confidence)
                    else:
                      print("Low confidence prediction. Skipping display.")

                    # Ensure valid display
                    if len(imgWhite.shape) == 2:
                       imgWhite = cv2.cvtColor(imgWhite, cv2.COLOR_GRAY2BGR)

                else:
                     print("Warning: imgWhite is empty or invalid.")
    # FPS calculation
    pTime = fps_calculation(img, pTime)

    cv2.imshow("Image", img)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q") or key == 27:
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()